In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [2]:
behaviors = pd.read_csv("../../../data/mind_small_train/behaviors_processed_small.csv")

In [3]:
behav = behaviors.to_numpy(copy=True)

In [4]:
behaviors['history'] = behaviors.history.str.split(' ')

In [5]:
behaviors.head()

,impression_id,user_id,time,history,impressions,length_history
0,1,U13740,11/11/2019 9:05:58 AM,"[N55189, N42782, N34694, N45794, N18445, N6330...",N55689-1 N35729-0,9
1,2,U91836,11/12/2019 6:11:30 PM,"[N31739, N6072, N63045, N23979, N35656, N43353...",N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,82
2,3,U73700,11/14/2019 7:01:48 AM,"[N10732, N25792, N7563, N21087, N41087, N5445,...",N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,16
3,4,U34670,11/11/2019 5:28:05 AM,"[N45729, N2203, N871, N53880, N41375, N43142, ...",N35729-0 N33632-0 N49685-1 N27581-0,10
4,6,U19739,11/11/2019 6:52:13 PM,"[N39074, N14343, N32607, N32320, N22007, N442,...",N21119-1 N53696-0 N33619-1 N25722-0 N2869-0,36


In [6]:
behav.shape

(136120, 6)

In [7]:
articles = []
for row in behav:
    for article in row[3].split(' '):
        articles.append(article)
        
    for article in row[4].split(' '):
        articles.append(article[:-2])

    
    

In [8]:
unique_articels = set(articles)

In [9]:
num_articles = len(unique_articels)

In [10]:
article_idx = {u:i for i, u in enumerate(unique_articels)}

In [11]:
behaviors['history_int'] = behaviors.history

In [12]:
behaviors['history_int'] = behaviors['history_int'].apply(lambda x: [article_idx[i] for i in x[-5:]])

In [13]:
behaviors.history_int[:3]

0      [38715, 47901, 15899, 15531, 222]
1     [45645, 27408, 12152, 4008, 28171]
2    [23414, 16556, 15800, 15952, 17047]
Name: history_int, dtype: object

In [14]:
articles_as_int = behaviors['history_int'].to_numpy()

In [15]:
behaviors['impressions_int'] = behaviors.impressions.str.split(' ')

In [16]:
behaviors['impressions_int_1'] = behaviors['impressions_int'].apply(lambda x: [article_idx[i[:-2]] for i in x if i[-1] == '1'])

In [17]:
impressions_1 = behaviors['impressions_int_1'].to_numpy()

In [18]:
behaviors['impressions_int_0'] = behaviors['impressions_int'].apply(lambda x: [article_idx[i[:-2]] for i in x if i[-1] == '0'])

In [19]:
impressions_0 = behaviors['impressions_int_0'].to_numpy()

In [20]:
complete_list_1s = []
for i in range(len(articles_as_int)):
    for art_int in articles_as_int[i]:
        complete_list_1s.append(art_int)
    complete_list_1s.append(impressions_1[i][0])
    


In [21]:
len(complete_list_1s)/6

136120.0

In [22]:
target_1s = [1]*136120

In [23]:
complete_list_0s = []
for i in range(len(articles_as_int)):
    for art_int in articles_as_int[i]:
        complete_list_0s.append(art_int)
    complete_list_0s.append(impressions_0[i][0])
    

In [24]:
len(complete_list_0s) /6

136120.0

In [87]:
len(complete_list_1s)/4 *3

612540.0

In [89]:
complete_list_1s_train = complete_list_1s[:612540]
complete_list_1s_test = complete_list_1s[612540:]

complete_list_0s_train = complete_list_0s[:612540]
complete_list_0s_test = complete_list_0s[612540:]

In [96]:
target_1s_train = [1]*(int(len(complete_list_1s_train)/6))
target_1s_test = [1]*(int(len(complete_list_1s_test)/6))

target_0s_train = [0]*(int(len(complete_list_0s_train)/6))
target_0s_test = [0]*(int(len(complete_list_0s_test)/6))

In [98]:
complete_list_1s_train = np.array(complete_list_1s_train)
complete_list_1s_test = np.array(complete_list_1s_test)

complete_list_0s_train = np.array(complete_list_0s_train)
complete_list_0s_test = np.array(complete_list_0s_test)

In [100]:
target_1s_train = np.array(target_1s_train)
target_1s_test = np.array(target_1s_test)

target_0s_train = np.array(target_0s_train)
target_0s_test = np.array(target_0s_test)

In [103]:
train = complete_list_1s_train + complete_list_0s_train
test = complete_list_1s_test + complete_list_0s_test

targets_train = target_1s_train + target_0s_train
targets_test = target_1s_test + target_0s_test

In [105]:
int(len(train) / 6)

102090

In [ ]:
train = train.reshape(int(len(train) / 6), 6,1)

In [109]:
test = test.reshape(int(len(test)/6),6,1)

In [111]:
len(train) == len(targets_train),len(test)== len(targets_test)

(True, True)

In [112]:
trajectories_to_score = test[:int(len(test)/2)]

In [115]:
trajectories_to_score.shape, test.shape[0]/2

((17015, 6, 1), 17015.0)

In [ ]:
target_0s = [0]*136120

In [26]:
complete_list = complete_list_1s + complete_list_0s

In [27]:
targets = target_1s + target_0s

In [30]:
targets[:3], complete_list[:10]

([1, 1, 1],
 [38715, 47901, 15899, 15531, 222, 31334, 45645, 27408, 12152, 4008])

In [31]:
targets = np.array(targets)

In [41]:
complete_list = np.array(complete_list)

In [42]:
complete_list = complete_list.reshape(behav.shape[0]*2,6,1 )

In [44]:
complete_list[:3]

array([[[38715],
        [47901],
        [15899],
        [15531],
        [  222],
        [31334]],

       [[45645],
        [27408],
        [12152],
        [ 4008],
        [28171],
        [47133]],

       [[23414],
        [16556],
        [15800],
        [15952],
        [17047],
        [48420]]])

In [47]:
len(targets) == len (complete_list)

True

In [84]:
k='anna'
k[1:], k[:1]

('nna', 'a')

In [51]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=num_articles, output_dim=64))
model.add(layers.LSTM(128))
model.add(layers.Dense(10))
model.add(layers.Dense(1,activation='sigmoid'))

In [52]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          3189696   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 3,289,813
Trainable params: 3,289,813
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['AUC'])

In [54]:
history = model.fit(complete_list, targets, epochs=3, validation_split=0.2, verbose=1)

Epoch 1/3
6806/6806 [==============================] - 218s 32ms/step - loss: 0.6354 - auc: 0.6467 - val_loss: 0.9147 - val_auc: 0.0000e+00
Epoch 2/3
6806/6806 [==============================] - 217s 32ms/step - loss: 0.6216 - auc: 0.6837 - val_loss: 0.9491 - val_auc: 0.0000e+00
Epoch 3/3
6806/6806 [==============================] - 215s 32ms/step - loss: 0.6189 - auc: 0.6891 - val_loss: 0.9340 - val_auc: 0.0000e+00


In [55]:
model_2 = keras.Sequential()
model_2.add(layers.Embedding(input_dim=num_articles, output_dim=64))
model_2.add(layers.GRU(128))
model_2.add(layers.Dense(10))
model_2.add(layers.Dense(1,activation='sigmoid'))

In [56]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          3189696   
_________________________________________________________________
gru (GRU)                    (None, 128)               74496     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 3,265,493
Trainable params: 3,265,493
Non-trainable params: 0
_________________________________________________________________


In [60]:
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['AUC'])

In [61]:
history = model_2.fit(complete_list, targets, epochs=3, validation_split=0.2, verbose=1)

Epoch 1/3
6806/6806 [==============================] - 220s 32ms/step - loss: 0.6316 - auc: 0.6580 - val_loss: 0.9354 - val_auc: 0.0000e+00
Epoch 2/3
6806/6806 [==============================] - 224s 33ms/step - loss: 0.6215 - auc: 0.6844 - val_loss: 0.9350 - val_auc: 0.0000e+00
Epoch 3/3
6806/6806 [==============================] - 224s 33ms/step - loss: 0.6192 - auc: 0.6892 - val_loss: 0.9300 - val_auc: 0.0000e+00


In [71]:
positive_courses = np.array(complete_list_1s)

In [73]:
len(positive_courses)/6

136120.0

In [74]:
positive_courses = positive_courses.reshape(136120, 6,1)

In [78]:
for course in positive_courses:
    
    
    
    print(course)
    break

[[38715]
 [47901]
 [15899]
 [15531]
 [  222]
 [31334]]
